# How does climate change feel around the globe?
# The final project from Spiced Academy
# Notebook for data cleaning and analysis

Check vizzes.ipynb for generating visualizations.

## Contents

[The questions](#questions)

[Importing libraries and packages](#import)

[Cleaning OECD data: Population exposure to extreme temperatures](#clean_oecd_1)

[Cleaning OECD data: Historical populations of countries](#clean_oecd_2)

[Cleaning UN data: GDP per capita](#clean_un)

[Cleaning NASA data: temperature anomaly](#clean_nasa)

[Correlation between population exposure and time](#corr_pop_time)

[Exposure change for the whole world](#exp_world)

[Temperature vs exposure worldwide](#temp_exp)

[The effects in rich and poor countries (GDP per capita)](#poor_rich)

## The questions <a id='questions'></a>

In this project, I attempt to answer these pressing questions related to periods of heat which are ever more frequent in virtually every part of the wolrd:
1. What percentage of people has direct experience with extreme heat?
2. Does this number change over time?
3. How much is it related to the global temperature anomaly?
4. Is there a clear link between wealth and heat exposure of populations?

## Importing libraries and packages <a id='import'></a>

In [2]:
import pandas as pd   # df workflow

import pycountry   # to get a dict between country names and alpha3 codes

# for clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from scipy import stats   # for A/B test

## Cleaning OECD data: Population exposure to extreme temperatures <a id='clean_oecd_1'></a>

We load the data about population exposures and perform cleaning for further analysis and integration with other datasets.

Years included: 1979 - 2021.

We will be interested mainly in four measures, describing the population exposure, although there is more measures in the dataset.

- 'HD_TN_POP_IND' gives the exposures to hot summer days (over 35 °C) AND tropical nights (over 20 °C).

- 'HD_POP_IND' is the exposure to hot summer days.

- 'TN_POP_IND' is the exposure to tropical nights.

- 'ID_POP_IND' is the exposure to icing days (doesn't exceed 0 °C).

In [183]:
df_exp = pd.read_csv('../data/oecd_population_exposure_to_extreme_temp.csv')

Choose only the relevant columns.

Reference area for individual countries (contains also provinces and greater regions of the world), measure (later we choose population exposure), duration (how many weeks per year did the phenomenon last), time period (from which year is the data), observed value.

In [184]:
df_exp = df_exp[['REF_AREA','MEASURE','DURATION','TIME_PERIOD','OBS_VALUE']]

Rename the columns to standard format, drop possible empty spaces.

In [185]:
df_exp.columns = df_exp.columns.str.lower().str.strip()

When reference area contains provinces, the abbreviation of a respective country ends with numerical characters. We are not interested in provinces and they occupy big part of the data frame, drop these lines.

In [186]:
df_exp = df_exp.loc[~(df_exp['ref_area'].str.endswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')))]
df_exp.reset_index(drop=True, inplace=True)

Some tropical countries reported zero population exposures. This data is most likely flawed and we drop these countries.

In [187]:
df_exp = df_exp[~df_exp['ref_area'].isin(['BDI', 'RWA', 'GNQ', 'PRI', 'BHS'])]
df_exp.reset_index(drop=True, inplace=True)

Now let us create a column with country names based on the alpha3 abbreviations.

First, make a dictionary to translate countries alpha3 abbreviations.

In [188]:
country_alpha3 = {}
for country in pycountry.countries:
    country_alpha3[country.alpha_3] = country.name

Apply to create a new column with country names.

In [189]:
df_exp['country'] = df_exp['ref_area'].map(country_alpha3)   # new column with country names
df_exp = df_exp[['ref_area', 'country', 'measure', 'duration', 'time_period', 'obs_value']]   # reorder columns

Optional: save the data frame into a file.

In [190]:
# df_exp.to_csv('../exported_dfs/exposures_all_durations.csv', index=False)

The data frame before, with separate durations, is useful for example for investigating trends in greater detail.

**For some purposes, we will be using the sum over all durations. Let us make such a grouped data frame.**

This can be done only for measure expressing percentages of population, so we will filter tha data frame for those first.

In [191]:
df_exp_summed = df_exp[df_exp['measure'].isin(['HD_POP_IND', 'TN_POP_IND', 'HD_TN_POP_IND', 'ID_POP_IND'])]

Now it is reasonable to rename the column 'obs_value' to 'exposure'.

In [192]:
df_exp_summed.rename(columns={'obs_value':'exposure'}, inplace=True)

C:\Users\radek\AppData\Local\Temp\ipykernel_24124\3465930645.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



We can group and sum up exposures over all durations.

In [193]:
df_exp_summed = df_exp_summed.groupby(['ref_area', 'country', 'measure', 'time_period'], as_index=False)['exposure'].sum()

The result sometimes slightly exceeds 100 % (up to 104.5 %). In the case, round the summed exposure to 100 %.

In [194]:
df_exp_summed.loc[df_exp_summed['exposure']>100, 'exposure'] = 100

Optional: save the data frame into a file.

In [195]:
# df_exp_summed.to_csv('../exported_dfs/exposures_summed.csv', index=False)

## Cleaning OECD data: Historical populations of countries <a id='clean_oecd_2'></a>

We load the data about population of countries in the previous years and perform cleaning for further analysis and integration with other datasets.

Years included: 1950 - 2021.

In [196]:
df_pop = pd.read_csv('../data/oecd_historical_population_data.csv')

Choose only the relevant columns.

Reference area (country), measure (population), sex and age (to filter fot total values later), time period (year of the observation), observed value.

In [197]:
df_pop = df_pop.drop(['DATAFLOW', 'TIME_HORIZ', 'OBS_STATUS', 'UNIT_MULT', 'DECIMALS'], axis='columns')

Rename the columns to standard format, drop possible empty spaces.

In [198]:
df_pop.columns = df_pop.columns.str.lower().str.strip()
df_pop.rename(columns={'obs_value':'population'}, inplace=True)   # more informative name

Keep only rows for total population across sexes and ages and drop the unnecessary columns after.

In [199]:
df_pop = df_pop[(df_pop['measure']=='POP') & (df_pop['unit_measure']=='PS') & (df_pop['sex']=='_T') &\
                (df_pop['age']=='_T')]
df_pop.reset_index(drop=True, inplace=True)
df_pop = df_pop.drop(['measure', 'unit_measure', 'sex', 'age'], axis='columns')

Optional: save the data frame into a file.

In [200]:
# df_pop.to_csv('../exported_dfs/populations_clean.csv', index=False)

## Cleaning UN data: GDP per capita <a id='clean_un'></a>

We load the data about GDP per capita of countries in the previous years and perform cleaning for further analysis and integration with other datasets.

Years included: 2017 - 2021.


In [201]:
df_gdp = pd.read_csv('../data/un_gdp_per_capita_current_prices.csv')

Choose only the relevant columns and rename.

In [202]:
df_gdp = df_gdp.drop('Item', axis='columns')
df_gdp.columns = ['country', 'time_period', 'gdp']

Change 'gdp' column type to float

In [203]:
df_gdp['gdp'] = df_gdp['gdp'].astype('float')

We are using alpha3 abbreviations of countries to merge different data frames but these are not present in the UN data.

Create an inversed dictionary (see cleaning OECD data) to translate countries in the UN dataset to alpha3 abbreviations.

In [204]:
country_alpha3_inversed = {}
for country in pycountry.countries:
    country_alpha3_inversed[country.name] = country.alpha_3

Some country names in the UN data frame are different than in the pycountries library, we need to rename them.

In [205]:
old_keys = ['China', 'United Kingdom', 'Turkey', "Korea, Democratic People's Republic of", "Korea, Republic of",\
           'Venezuela, Bolivarian Republic of', 'Bolivia, Plurinational State of',\
            'Congo, The Democratic Republic of the', 'Tanzania, United Republic of', 'Moldova, Republic of',\
            'North Macedonia']
new_keys = ['China (mainland)', 'United Kingdom of Great Britain and Northern Ireland', 'Türkiye',\
           "Democratic People's Republic of Korea", "Republic of Korea", 'Venezuela (Bolivarian Republic of)',\
           'Bolivia (Plurinational State of)', 'Democratic Republic of the Congo',\
            'United Republic of Tanzania: Mainland', 'Republic of Moldova', 'Republic of North Macedonia']

for key in range(len(new_keys)):
    old_key = old_keys[key]
    new_key = new_keys[key]
    country_alpha3_inversed[new_key] = country_alpha3_inversed.pop(old_key)

Create the column with alpha3 abbreviations.

In [206]:
df_gdp['ref_area'] = df_gdp['country'].map(country_alpha3_inversed)
df_gdp = df_gdp[['ref_area', 'country', 'time_period', 'gdp']]   # redorder columns

Optional: save the data frame into a file.

In [207]:
# df_gdp.to_csv('../exported_dfs/gdp_clean.csv', index=False)

## Cleaning NASA data: temperature anomaly <a id='clean_nasa'></a>

We load the data about temperature anomaly and perform cleaning for further analysis and integration with other datasets.

Years included: 1880 - 2023.

Monthly temperature anomaly, compared with the mean temperature from 1951-1980.

In [208]:
df_temp_anomaly = pd.read_csv('../data/nasa_global_mean_temperature_anomaly.csv', skiprows=1)

Choose only relevant columns (year and monthly anomalies) and rename.

In [209]:
df_temp_anomaly = df_temp_anomaly.iloc[:,0:13]
df_temp_anomaly.columns = df_temp_anomaly.columns.str.lower().str.strip()

Choose only the relevant years (rows).

In [210]:
df_temp_anomaly = df_temp_anomaly[df_temp_anomaly['year'].between(1979,2021)]
df_temp_anomaly = df_temp_anomaly.reset_index(drop=True)

Change data type to floats to perform calculations.

In [211]:
df_temp_anomaly[df_temp_anomaly.columns[1:13]] = df_temp_anomaly[df_temp_anomaly.columns[1:13]].astype(float)

Add a column with average anomalies for every year and keep only these two columns.

In [212]:
df_temp_anomaly['avg_anomaly'] = df_temp_anomaly.iloc[:, 1:].mean(axis=1)
df_temp_anomaly = df_temp_anomaly[['year', 'avg_anomaly']]

Optional: save the data frame into a file.

In [213]:
# df_temp_anomaly.to_csv('../exported_dfs/temp_anomaly_clean.csv', index=False)

## Correlation between population exposure and time <a id='corr_pop_time'></a>

We are going to calculate correlation between population exposure and time for individual countries.

Read the cleaned data frame from a file or use df_exp from above.

In [214]:
# df_analyze_corr = pd.read_csv('../exported_dfs/exposures_summed.csv')
df_analyze_corr = df_exp_summed

Keep only the lines where 'measure' contains data about population exposures.

In [215]:
df_analyze_corr =\
df_analyze_corr.loc[~(df_analyze_corr['measure'].str.contains('TEMP') |\
                      df_analyze_corr['measure'].str.contains('UTCI_POP_IND'))]
df_analyze_corr.reset_index(drop=True, inplace=True)

We need a sum of exposures for all durations, therefore group by reference area, measure and time period and calculate the sum.

In [216]:
df_analyze_corr = df_exp_summed.groupby(['ref_area', 'measure', 'time_period'], as_index=False)['exposure'].sum()

**The actual correlations.**

Below, select the measure in which we are interested.

In [217]:
df_analyze_corr = df_analyze_corr[df_analyze_corr['measure']=='HD_TN_POP_IND']   # HERE SELECT THE MEASURE
df_analyze_corr = df_analyze_corr.drop('measure', axis='columns')   # do not need the column any longer

A loop, calculating correlation coefficient for each country.

In [218]:
countries = list(df_analyze_corr['ref_area'].unique())   # list of countries present
corr_coeffs = []   # empty list for correlation coefficients

for country in countries:
    df_country = df_analyze_corr[df_analyze_corr['ref_area']==country]   # filter for the country
    df_corr = df_country.corr(numeric_only=True)   # calculate correlation coefficients for numeric columns
    coeff = df_corr.iloc[0,1]   # choose the relevant number
    corr_coeffs.append(coeff)   # store the coefficient inside of the list

We can store the result in a data frame.

In [219]:
df_corr_results = pd.DataFrame(columns=['countries','corr_coeff'])
df_corr_results['countries'] = countries
df_corr_results['corr_coeff'] = corr_coeffs

Optional: save the data frame into a file.

In [220]:
# df_corr_results.to_csv('../exported_dfs/correlations.csv', index=False)

## Exposure change for the whole world <a id='exp_world'></a>

Bonus (not included in the graduation presentation): If we want to describe the trends of population exposures worldwide, we need to calculate weighted average over all countries for every year.

Read the cleaned data frame from a file or use df_exp_summed from above.

In [221]:
df_exp_summed = pd.read_csv('../exported_dfs/exposures_summed.csv')

Select the measure we are interested in.

In [222]:
df_exp_measure = df_exp_summed[df_exp_summed['measure']=='HD_TN_POP_IND'] # HERE SELECT THE MEASURE

Merge data frames with exposure and population size.

In [223]:
df_exp_pop = pd.merge(left=df_exp_measure, right=df_pop, how='left', on=['ref_area', 'time_period'])

For further calculations, we need to replace missing values with zeros.

In [224]:
df_exp_pop['population'] = df_exp_pop['population'].fillna(0)

A loop, calculating weighted average for each year.

In [225]:
# new df for years and weighted averages:

years = list(df_exp_pop['time_period'].unique())   # list of years present
exp_world = []   # empty list for worldwide exposures

for year in years:
    df_calc = df_exp_pop[df_exp_pop['time_period']==year]   # filter for the year
    avg_world = sum(df_calc['exposure'] * df_calc['population'])/ sum(df_calc['population'])   # weighted average
    exp_world.append(avg_world)   # store the average inside of the list

We can store the result in a data frame.

In [226]:
df_world_exp = pd.DataFrame(columns=['year','exposure'])
df_world_exp['year'] = years
df_world_exp['exposure'] = exp_world

Optional: save the data frame into a file.

In [227]:
# df_world_exp.to_csv('../exported_dfs/world_exp.csv', index=False)

## Temperature vs exposure worldwide <a id='temp_exp'></a>

We will merge the data on worldwide population exposure and temperature anomalies to check for correlation and to visualize any trends.

Read the cleaned data frames from files or use df_temp_anomaly and df_world_exp from above.

In [171]:
# df_temp_anomaly = pd.read_csv('../exported_dfs/temp_anomaly_clean.csv')
# df_world_exp = pd.read_csv('../exported_dfs/world_exp.csv')

Merge temperature anomaly and worldwide population exposure data frames.

In [228]:
df_temp_anomaly_world_exp = pd.merge(df_temp_anomaly, df_world_exp)

Calculate the correlation coefficients.

In [229]:
df_temp_anomaly_world_exp.corr()

,year,avg_anomaly,exposure
year,1.000000,0.929837,0.915518
avg_anomaly,0.929837,1.000000,0.930867
exposure,0.915518,0.930867,1.000000


## The effects in rich and poor countries (GDP per capita) <a id='poor_rich'></a>

We will investigate, if the population exposure is in general greater in poor countries. We will first calculate correlation between population exposure and GDP per capita and then also use k-means to cluster countries using these two variables. For two clusters, we will also perform an A/B test to compare the group characteristics.

**Correlation between population exposure and GDP per capita**

Read the cleaned data frames from files or use df_exp_summed, df_gdp from above.

In [238]:
df_exp_summed = pd.read_csv('../exported_dfs/exposures_summed.csv')
df_gdp = pd.read_csv('../exported_dfs/gdp_clean.csv')

First, select the measure we are interested in.

In [239]:
df_exp_summed = df_exp_summed[df_exp_summed['measure']=='HD_TN_POP_IND'] # HERE SELECT THE MEASURE
df_exp_summed = df_exp_summed.drop('measure', axis='columns') # do not need the column any longer

Merge the data frames for population exposures and GDP per capita. Both data frames contain 'country' column but some names are slightly different, drop them first.

In [240]:
df_exp_summed.drop('country', axis='columns', inplace=True)
df_gdp.drop('country', axis='columns', inplace=True)
df_exp_gdp = pd.merge(df_exp_summed, df_gdp, how='inner', on=['ref_area', 'time_period'])

The correlation between numerical variables.

In [241]:
df_exp_gdp.corr(numeric_only=True)

,time_period,exposure,gdp
time_period,1.000000,-0.004992,0.013863
exposure,-0.004992,1.000000,-0.317923
gdp,0.013863,-0.317923,1.000000


**k-means**

The algorithm gives more insightful results when omitting small countries with extremely high GDP per capita and zero population exposure. Drop these countries.

In [242]:
df_exp_gdp = df_exp_gdp[~df_exp_gdp['ref_area'].isin(['LUX', 'BMU', 'LIE', 'MCO'])]

Group by countries, get average population exposure and GDP per capita to cluster.

We can remember that GDP data is only available for the period 2017 - 2021. This is a reasonable time interval to explore, since the averaging supresses fluctuations but all the data is recent and describes the current situation.

In [243]:
df_exp_gdp_avg = df_exp_gdp.groupby('ref_area')[['exposure', 'gdp']].mean()

Create a data frame with numerical columns only.

In [244]:
df_num = df_exp_gdp_avg[['exposure', 'gdp']]

Standardize data (standard deviation of all columns 1, mean 0).

In [245]:
scaler = StandardScaler()
scaler.fit(df_num)
df_num_scaled = scaler.transform(df_num) # array of standardized data
df_standardized = pd.DataFrame(df_num_scaled, columns=df_num.columns) # data frame from the array

Optional: Elbow diagram to derive appropriate number of clusters.

In [44]:
# K = range(2, 10)   # try the fitting with up to 10 clusters
# inertia = []
# for k in K:
#     kmeans = KMeans(n_clusters=k,
#                     n_init=10)
#     kmeans.fit(df_standardized)
#     inertia.append(kmeans.inertia_)

# plt.figure(figsize=(16,8))
# plt.plot(K, inertia, 'bx-')
# plt.xlabel('k')
# plt.ylabel('inertia')
# plt.xticks(np.arange(min(K), max(K), 1.0))
# plt.title('Elbow Diagram')

k-Means (**choose number of clusters here**)

In [246]:
kmeans = KMeans(n_clusters=2) # CHOOSE NO OF CLUSTERS
kmeans.fit(df_standardized)

D:\DAsoft\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans(n_clusters=2)

Defining the clusters (aray of labels).

In [247]:
clusters = kmeans.predict(df_standardized)

Numbers of members in each cluster.

In [248]:
pd.Series(clusters).value_counts()

0    97
1    96
Name: count, dtype: int64

Adding clusters to the original data frame.

In [249]:
df_clustered = df_exp_gdp_avg.copy()   # copy of the original data frame
df_clustered['cluster'] = clusters   # new column with cluster labels
df_clustered['cluster'] = df_clustered['cluster'].astype(str)   # labels as string, useful for visualizations
df_clustered = df_clustered.reset_index()

Optional: save the data frame into a file.

In [251]:
# df_clustered.to_csv('../exported_dfs/clusters.csv', index=False)

**For 2 clusters: A/B test**

Making separate data frames for each custer to compare.

In [252]:
df_cluster_0 = df_clustered[df_clustered['cluster']=='0']
df_cluster_1 = df_clustered[df_clustered['cluster']=='1']

Two-tailed A/B test, p-value.

In [253]:
test_statistic, pvalue = stats.ttest_ind(df_cluster_0['gdp'], df_cluster_1['gdp'])
print ('GDP:', test_statistic, pvalue)

test_statistic, pvalue = stats.ttest_ind(df_cluster_0['exposure'], df_cluster_1['exposure'])
print ('Exposure:', test_statistic, pvalue)

GDP: -7.229990853307366 1.1314951828134623e-11
Exposure: 20.573017749248752 2.468916104500439e-50
